# Setup

This notebook contains the script to create the necessary Azure environment to run the provided samples. The notebook uses [PowerShell](https://learn.microsoft.com/powershell/scripting/install/installing-powershell?view=powershell-7.4) and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

This notebook performs two tasks:

- Deployment of an Azure OpenAI instance with deployed gpt-4o model to run the provided samples
- Store all necessary service endpoints, service API keys and Azure OpenAI deployment names in a **centralized env file** (./configuration/application.env). 
  
If you already have instances of Azure OpenAI with a deployed chat completion model can rename the [configuration template](../conf/application.env.example) to `application.env` and provide endpoint, API key and deployment name of a chat completion model.I suggest to run the notebook to start a clean environment.

## Step 1:   Login to Azure; Get, Set subscription

In [ ]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($loggedIn -ne $null) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}
# Retrieve default subscription id
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set Subscription
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

## Step 2:   Define project unifier

The project unifier is used to allow multiple deployments of services which have a need for a unique custom endpoint.

In [ ]:
# Define Project Unifier

$random = Get-Random -Minimum 100 -Maximum 999

Write-Host "Unifier set to: $random"

## Step 3:   Create Resource Group

In this sample all resources are deployed to `swedencentral`. Feel free to change to your preferred location.

In [ ]:
# Create Resource Group
$resourceGroup = "OpenAI.FileTool"
$location = "swedencentral"

az group create `
    --location $location `
    --resource-group $resourceGroup

## Step 4:   Create Azure OpenAI instance

An instance of Azure Cognitive Service with the kind `OpenAI` will be created. The `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
# Create Azure OpenAI Account
$csOpenAIName = "FT_AOAI_$random"

az cognitiveservices account create `
    --name $csOpenAIName `
    --resource-group $resourceGroup `
    --location $location `
    --kind OpenAI `
    --sku S0 `
    --yes

$csOpenAIId = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query id `
        --output tsv `
)

$csOpenAIEndpoint = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query properties.endpoint `
        --output tsv `
)

$csOpenAIApiKey = (
    az cognitiveservices account keys list `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query key1 `
        --output tsv `
)


## Step 5:   Deploy Azure OpenAI models

Two LLM models are deployed to the newly created Azure Cognitive Service instance: 

- A chat completion model. In the sample we're deploying `gpt-4o`. 

In [ ]:
# Chat Completion Model
$modelChatCompletionDeploymentName = "gpt-4o"
$modelName = "gpt-4o"
$modelVersion = "2024-05-13"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelChatCompletionDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1


## Step 6: Create application.env file

In [ ]:
# Store configuration 
$configurationFile = "../configuration/application.env"
New-Item -Name $configurationFile -ItemType File -Force


function Set-ConfigurationFileVariable($configurationFile, $variableName, $variableValue) {
    if (Select-String -Path $configurationFile -Pattern $variableName) {
        (Get-Content $configurationFile) | Foreach-Object {
            $_ -replace "$variableName = .*", "$variableName = $variableValue"
        } | Set-Content $configurationFile
    } else {
        Add-Content -Path $configurationFile -value "$variableName = $variableValue"
    }
}

Set-ConfigurationFileVariable $configurationFile "ST_AOAI_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "ST_AOAI_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "ST_CHATCOMPLETION_DEPLOYMENTNAME" $modelChatCompletionDeploymentName
Set-ConfigurationFileVariable $configurationFile "ST_ASSETS_FOLDER" "../assets"

Write-Host "Configuration file created at: $configurationFile"
